In [1]:
import plotly.io as pio
# pio.renderers.default = 'notebook'
pio.renderers.default = 'jupyterlab'

In [3]:
import itertools
import ipywidgets as widgets
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipycanvas import Canvas, MultiCanvas
import plotly.io as pio

# Ensure Plotly renders in Jupyter Notebook/Lab
pio.renderers.default = 'notebook'

# Canvas settings
width, height = 900, 600
horizontal_padding = 300
vertical_padding = 100

obs_to_angle_sensitivity = 0.25
tx_power = 2e3

sensors_fill_color = 'red'
position_fill_color = 'green'

n_sensors = 4
sensors_radius = 19

def obs(x_1: int, y_1: int, x_sensor: int, y_sensor) -> float:
    """Computes an observation given the x and y coordinates for two different positions."""
    return tx_power / np.sqrt((x_1 - x_sensor)**2 + (y_1 - y_sensor)**2)

def obs_to_angle(obs: float) -> float:
    """Turns a measure into an angle in radians."""
    return 2*np.pi * ((1. / (1. + np.exp(-obs_to_angle_sensitivity*obs))) - 0.5) / (1 - 0.5)

# Compute sensor positions
n_sensors_per_axis = int(np.sqrt(n_sensors))
sensors_x_coordinates = np.linspace(horizontal_padding, width-horizontal_padding, n_sensors_per_axis)
sensors_y_coordinates = np.linspace(vertical_padding, height-vertical_padding, n_sensors_per_axis)
sensors_coordinates = list(itertools.product(sensors_x_coordinates, sensors_y_coordinates))
n_sensors = len(sensors_coordinates)

target_position = []
observations = []

# Create FigureWidget for real-time updates
fig_widget = go.FigureWidget(make_subplots(
    rows=n_sensors_per_axis, cols=n_sensors_per_axis,
    subplot_titles=[f"Sensor {i+1}" for i in range(n_sensors)]
))

# Create scatter plots for each sensor
plots = []
for i in range(n_sensors):
    row, col = divmod(i, n_sensors_per_axis)
    trace = go.Scatter(x=[], y=[], mode='lines', name=f'Sensor {i+1}')
    fig_widget.add_trace(trace, row=row+1, col=col+1)
    plots.append(trace)

# Update layout
fig_widget.update_layout(
    height=600, width=800,
    title_text="Sensor Observations",
    showlegend=False
)

# Canvas setup
canvas = MultiCanvas(n_canvases=2, width=width, height=height)
canvas[0].fill_style = sensors_fill_color
canvas[0].stroke_style = 'blue'
canvas[0].stroke_rect(0, 0, canvas[0].width, canvas[0].height)

# Draw sensors
for x, y in sensors_coordinates:
    canvas[0].stroke_arc(x, y, 20, 0, 2*np.pi)

# Mouse click event handler
def handle_mouse_up(x, y):
    """Handles mouse clicks to update target position and sensor observations."""
    print("Click detected at:", x, y)  # Debugging

    target_position.append((x, y))
    canvas[1].fill_style = position_fill_color
    canvas[1].fill_rect(x, y, 10, 10)

    new_observations = [obs(x, y, sx, sy) for sx, sy in sensors_coordinates]
    observations.append(new_observations)
    observations_np = np.array(observations)

    # Update plots
    with fig_widget.batch_update():
        for i, trace in enumerate(fig_widget.data):
            trace.x = np.arange(observations_np.shape[0])
            trace.y = observations_np[:, i]

canvas[1].on_mouse_up(handle_mouse_up)

# Display everything
display(fig_widget)
canvas


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Sensor 1',
              'type': 'scatter',
              'uid': '96edf423-79c4-4d7b-be6a-1bb31b676cba',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'mode': 'lines',
              'name': 'Sensor 2',
              'type': 'scatter',
              'uid': '090c6a2e-3d13-4a32-a51f-1af9d03cd40b',
              'x': [],
              'xaxis': 'x2',
              'y': [],
              'yaxis': 'y2'},
             {'mode': 'lines',
              'name': 'Sensor 3',
              'type': 'scatter',
              'uid': 'fb9dc33b-131a-4920-961a-07ee630b7a7b',
              'x': [],
              'xaxis': 'x3',
              'y': [],
              'yaxis': 'y3'},
             {'mode': 'lines',
              'name': 'Sensor 4',
              'type': 'scatter',
              'uid': 'eee58075-98b4-4571-a186-b30eb0cc9312',
              'x': [],
             

MultiCanvas(height=600, width=900)